In [1]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [ ]:
from data_retrieval import lipade_groundtruth
import cv2 as cv
import concurrent
import numpy as np

corpus = "lipade_groundtruth"
resultsPath = "../results/distance/" + corpus + "/"

In [ ]:
x,_,y = lipade_groundtruth.getDataset()
for i in range(len(x)):
    x[i] = cv.imread(x[i], cv.IMREAD_GRAYSCALE)

In [ ]:
sift = cv.SIFT_create()

def computeSiftDescriptors(i,x):
    kp, des = sift.detectAndCompute(x[i], None)
    return i, kp, des

keypoints = [0 for _ in range(len(x))]
descriptors = [0 for _ in range(len(x))]

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(computeSiftDescriptors, i, x) for i in range(len(x))]
    for future in concurrent.futures.as_completed(futures):
        i,kp,des = future.result()
        keypoints[i] = kp
        descriptors[i] = des

Proportional version :

In [ ]:
match_score = np.zeros((len(x), len(x)))

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)

def computeMatchScore(i,j,descriptors,flann):
    # 0 -> No match
    # 1 -> Maximal number of match (= number of descriptors of the image with the smallest amount of descriptors)
    matches = flann.knnMatch(descriptors[i], descriptors[j], k=2)
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]
    score = min(1, len(good) / min(len(descriptors[i]), len(descriptors[j]))) # proportional
    return i,j,score

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(computeMatchScore, i, j, descriptors, flann) for i in range(len(x)) for j in range(i + 1, len(x))]
    for future in concurrent.futures.as_completed(futures):
        i,j,score = future.result()
        match_score[i,j] = score

match_score = match_score + match_score.T

# proportional
match_score += np.identity(match_score.shape[0])
np.save(resultsPath + "sift_proportional.npy", 1 - match_score) # distance

Absolute version :

In [ ]:
match_score = np.zeros((len(x), len(x)))

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)

def computeMatchScore(i,j,descriptors,flann):
    # 0 -> No match
    # 1 -> Maximal number of match (= number of descriptors of the image with the smallest amount of descriptors)
    matches = flann.knnMatch(descriptors[i], descriptors[j], k=2)
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]
    score = len(good) # absolute
    return i,j,score

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(computeMatchScore, i, j, descriptors, flann) for i in range(len(x)) for j in range(i + 1, len(x))]
    for future in concurrent.futures.as_completed(futures):
        i,j,score = future.result()
        match_score[i,j] = score

match_score = match_score + match_score.T

# absolute
match_score += np.identity(match_score.shape[0]) * (2 * np.max(match_score)) # to ensure that the first result is himself
match_score /= np.max(match_score)
np.save(resultsPath + "sift_absolute.npy", 1 - match_score) # distance